# Book Recommendation System

## Import all Packages 

In [1]:
import pandas as pd
import numpy as np
import pandas as pd
import nltk
import json
import re
import csv
from tqdm import tqdm
from textblob import TextBlob

## Open and try to read all contents

In [2]:

pd.set_option('display.max_colwidth', 300)

data = []

with open("../BOOK_DATASET/booksummaries.txt", 'r', encoding='utf-8') as f:
    reader = csv.reader(f, dialect='excel-tab')
    for row in tqdm(reader):
        data.append(row)


16559it [00:01, 9450.67it/s] 


## We append a all attributes in csv

In [3]:
book_index = []
book_id = []
book_author = []
book_name = []
summary = []
genre = []
a = 1
for i in tqdm(data):
    book_index.append(a)
    a = a+1
    book_id.append(i[0])
    book_name.append(i[2])
    book_author.append(i[3])
    genre.append(i[5])
    summary.append(i[6])

df = pd.DataFrame({'Index': book_index, 'ID': book_id, 'BookTitle': book_name, 'Author': book_author,
                       'Genre': genre, 'Summary': summary})
df.head()

100%|████████████████████████████████████████████████████████████████████████| 16559/16559 [00:00<00:00, 346723.04it/s]


,Index,ID,BookTitle,Author,Genre,Summary
0,1,620,Animal Farm,George Orwell,"{""/m/016lj8"": ""Roman \u00e0 clef"", ""/m/06nbt"": ""Satire"", ""/m/0dwly"": ""Children's literature"", ""/m/014dfn"": ""Speculative fiction"", ""/m/02xlf"": ""Fiction""}","Old Major, the old boar on the Manor Farm, calls the animals on the farm for a meeting, where he compares the humans to parasites and teaches the animals a revolutionary song, 'Beasts of England'. When Major dies, two young pigs, Snowball and Napoleon, assume command and turn his dream into a p..."
1,2,843,A Clockwork Orange,Anthony Burgess,"{""/m/06n90"": ""Science Fiction"", ""/m/0l67h"": ""Novella"", ""/m/014dfn"": ""Speculative fiction"", ""/m/0c082"": ""Utopian and dystopian fiction"", ""/m/06nbt"": ""Satire"", ""/m/02xlf"": ""Fiction""}","Alex, a teenager living in near-future England, leads his gang on nightly orgies of opportunistic, random ""ultra-violence."" Alex's friends (""droogs"" in the novel's Anglo-Russian slang, Nadsat) are: Dim, a slow-witted bruiser who is the gang's muscle; Georgie, an ambitious second-in-command; and..."
2,3,986,The Plague,Albert Camus,"{""/m/02m4t"": ""Existentialism"", ""/m/02xlf"": ""Fiction"", ""/m/0pym5"": ""Absurdist fiction"", ""/m/05hgj"": ""Novel""}","The text of The Plague is divided into five parts. In the town of Oran, thousands of rats, initially unnoticed by the populace, begin to die in the streets. A hysteria develops soon afterward, causing the local newspapers to report the incident. Authorities responding to public pressure order t..."
3,4,1756,An Enquiry Concerning Human Understanding,David Hume,,"The argument of the Enquiry proceeds by a series of incremental steps, separated into chapters which logically succeed one another. After expounding his epistemology, Hume explains how to apply his principles to specific topics. In the first section of the Enquiry, Hume provides a rough introdu..."
4,5,2080,A Fire Upon the Deep,Vernor Vinge,"{""/m/03lrw"": ""Hard science fiction"", ""/m/06n90"": ""Science Fiction"", ""/m/014dfn"": ""Speculative fiction"", ""/m/01hmnh"": ""Fantasy"", ""/m/02xlf"": ""Fiction""}","The novel posits that space around the Milky Way is divided into concentric layers called Zones, each being constrained by different laws of physics and each allowing for different degrees of biological and technological advancement. The innermost, the ""Unthinking Depths"", surrounds the galacti..."


### Checking with NULL values

In [4]:
df[df['Genre'] == ''].index

Int64Index([    3,     7,    10,    11,    12,    13,    14,    16,    17,
               18,
            ...
            16539, 16540, 16543, 16547, 16548, 16550, 16552, 16553, 16554,
            16557],
           dtype='int64', length=3718)

In [5]:
len(df[df['Genre'] == ''].index)

3718

In [6]:
df[df['Summary'] == ''].index

Int64Index([], dtype='int64')

## Checking null values and drop the empty 

In [7]:
df = df.drop(df[df['Genre'] == ''].index)
df = df.drop(df[df['Summary'] == ''].index)

genres_cleaned = []
for i in df['Genre']:
    genres_cleaned.append(list(json.loads(i).values()))
df['Genres'] = genres_cleaned


In [8]:
df.head(2)

,Index,ID,BookTitle,Author,Genre,Summary,Genres
0,1,620,Animal Farm,George Orwell,"{""/m/016lj8"": ""Roman \u00e0 clef"", ""/m/06nbt"": ""Satire"", ""/m/0dwly"": ""Children's literature"", ""/m/014dfn"": ""Speculative fiction"", ""/m/02xlf"": ""Fiction""}","Old Major, the old boar on the Manor Farm, calls the animals on the farm for a meeting, where he compares the humans to parasites and teaches the animals a revolutionary song, 'Beasts of England'. When Major dies, two young pigs, Snowball and Napoleon, assume command and turn his dream into a p...","[Roman à clef, Satire, Children's literature, Speculative fiction, Fiction]"
1,2,843,A Clockwork Orange,Anthony Burgess,"{""/m/06n90"": ""Science Fiction"", ""/m/0l67h"": ""Novella"", ""/m/014dfn"": ""Speculative fiction"", ""/m/0c082"": ""Utopian and dystopian fiction"", ""/m/06nbt"": ""Satire"", ""/m/02xlf"": ""Fiction""}","Alex, a teenager living in near-future England, leads his gang on nightly orgies of opportunistic, random ""ultra-violence."" Alex's friends (""droogs"" in the novel's Anglo-Russian slang, Nadsat) are: Dim, a slow-witted bruiser who is the gang's muscle; Georgie, an ambitious second-in-command; and...","[Science Fiction, Novella, Speculative fiction, Utopian and dystopian fiction, Satire, Fiction]"


## Summary Clean(Using Regular Expression)

In [9]:
def clean_summary(text):
    text = re.sub("\'", "", text)
    text = re.sub("[^a-zA-Z]"," ",text)
    text = ' '.join(text.split())
    text = text.lower()
    return text

df['clean_summary'] = df['Summary'].apply(lambda x: clean_summary(x))
df.head(2)

,Index,ID,BookTitle,Author,Genre,Summary,Genres,clean_summary
0,1,620,Animal Farm,George Orwell,"{""/m/016lj8"": ""Roman \u00e0 clef"", ""/m/06nbt"": ""Satire"", ""/m/0dwly"": ""Children's literature"", ""/m/014dfn"": ""Speculative fiction"", ""/m/02xlf"": ""Fiction""}","Old Major, the old boar on the Manor Farm, calls the animals on the farm for a meeting, where he compares the humans to parasites and teaches the animals a revolutionary song, 'Beasts of England'. When Major dies, two young pigs, Snowball and Napoleon, assume command and turn his dream into a p...","[Roman à clef, Satire, Children's literature, Speculative fiction, Fiction]",old major the old boar on the manor farm calls the animals on the farm for a meeting where he compares the humans to parasites and teaches the animals a revolutionary song beasts of england when major dies two young pigs snowball and napoleon assume command and turn his dream into a philosophy t...
1,2,843,A Clockwork Orange,Anthony Burgess,"{""/m/06n90"": ""Science Fiction"", ""/m/0l67h"": ""Novella"", ""/m/014dfn"": ""Speculative fiction"", ""/m/0c082"": ""Utopian and dystopian fiction"", ""/m/06nbt"": ""Satire"", ""/m/02xlf"": ""Fiction""}","Alex, a teenager living in near-future England, leads his gang on nightly orgies of opportunistic, random ""ultra-violence."" Alex's friends (""droogs"" in the novel's Anglo-Russian slang, Nadsat) are: Dim, a slow-witted bruiser who is the gang's muscle; Georgie, an ambitious second-in-command; and...","[Science Fiction, Novella, Speculative fiction, Utopian and dystopian fiction, Satire, Fiction]",alex a teenager living in near future england leads his gang on nightly orgies of opportunistic random ultra violence alexs friends droogs in the novels anglo russian slang nadsat are dim a slow witted bruiser who is the gangs muscle georgie an ambitious second in command and pete who mostly pla...


## TextBlob --- NLP Library

### We take a polarity variable in TextBlob library and calculate sentimental numerical value of summary and polarity range is (-1 to 1)

In [10]:
def get_emotion(text):
    analysis = TextBlob(text)
    polarity = analysis.sentiment.polarity
    if polarity > 0.3:
        return 'Excited'
    elif polarity > 0.1 and polarity <= 0.3:
        return 'Happy'
    elif polarity < -0.3:
        return 'Angry'
    elif polarity >= -0.3 and polarity < -0.1:
        return 'Sad'
    elif polarity >= -0.1 and polarity < 0.1:
        return 'Neutral'
    elif polarity >= 0.1 and polarity <= 0.3:
        return 'Content'
    elif polarity > 0.3 and polarity <= 0.5:
        return 'Enthusiastic'
    elif polarity > 0.5:
        return 'Surprised'
    elif polarity < -0.3 and polarity >= -0.5:
        return 'Disappointed'
    elif polarity < -0.5:
        return 'Frustrated'
    elif polarity >= 0.3 and polarity <= 0.5:
        return 'Proud'
    elif polarity < -0.1 and polarity >= -0.3:
        return 'Jealous'
    elif polarity >= 0.1 and polarity <= 0.3:
        return 'Hopeful'
    elif polarity < -0.5 and polarity >= -0.7:
        return 'Worried'
    elif polarity < -0.7:
        return 'Insecure'
    elif polarity > 0.5 and polarity <= 0.7:
        return 'Grateful'
    elif polarity > 0.7:
        return 'Calm'
    elif polarity < -0.7:
        return 'Lonely'
    elif polarity < 0:
        return 'Nervous'
    elif polarity > 0:
        return 'Excited'


df['emotion'] = df['clean_summary'].apply(lambda x: get_emotion(x))
df.head(2)


,Index,ID,BookTitle,Author,Genre,Summary,Genres,clean_summary,emotion
0,1,620,Animal Farm,George Orwell,"{""/m/016lj8"": ""Roman \u00e0 clef"", ""/m/06nbt"": ""Satire"", ""/m/0dwly"": ""Children's literature"", ""/m/014dfn"": ""Speculative fiction"", ""/m/02xlf"": ""Fiction""}","Old Major, the old boar on the Manor Farm, calls the animals on the farm for a meeting, where he compares the humans to parasites and teaches the animals a revolutionary song, 'Beasts of England'. When Major dies, two young pigs, Snowball and Napoleon, assume command and turn his dream into a p...","[Roman à clef, Satire, Children's literature, Speculative fiction, Fiction]",old major the old boar on the manor farm calls the animals on the farm for a meeting where he compares the humans to parasites and teaches the animals a revolutionary song beasts of england when major dies two young pigs snowball and napoleon assume command and turn his dream into a philosophy t...,Happy
1,2,843,A Clockwork Orange,Anthony Burgess,"{""/m/06n90"": ""Science Fiction"", ""/m/0l67h"": ""Novella"", ""/m/014dfn"": ""Speculative fiction"", ""/m/0c082"": ""Utopian and dystopian fiction"", ""/m/06nbt"": ""Satire"", ""/m/02xlf"": ""Fiction""}","Alex, a teenager living in near-future England, leads his gang on nightly orgies of opportunistic, random ""ultra-violence."" Alex's friends (""droogs"" in the novel's Anglo-Russian slang, Nadsat) are: Dim, a slow-witted bruiser who is the gang's muscle; Georgie, an ambitious second-in-command; and...","[Science Fiction, Novella, Speculative fiction, Utopian and dystopian fiction, Satire, Fiction]",alex a teenager living in near future england leads his gang on nightly orgies of opportunistic random ultra violence alexs friends droogs in the novels anglo russian slang nadsat are dim a slow witted bruiser who is the gangs muscle georgie an ambitious second in command and pete who mostly pla...,Neutral


### In above we calculate the emotion of all summaries

In [11]:
len(df)

12841

### Length of some Emotions

In [12]:
print(len(df[df['emotion']=='Neutral']))
print(len(df[df['emotion']=='Happy']))
print(len(df[df['emotion']=='Sad']))
print(len(df[df['emotion']=='Excited']))

8732
3103
669
223


### Again we using a same library to extract the Cleaned Genre<br> <br>Reason : There are lot of unwanted contents


In [13]:
from textblob import TextBlob

def get_genre(genres):
    blob = TextBlob(' '.join(genres).lower())
    horror = {"horror", "terror", "fear", "suspense", "supernatural"}
    fantasy = {"fantasy", "magic", "mythical", "legendary", "enchanted"}
    sci_fi = {"science fiction", "sci-fi", "space"}
    romance = {"romance", "love", "heartbreak", "passion"}
    mystery = {"mystery", "detective", "crime", "sleuth"}
    thriller = {"thriller", "suspense", "action"}
    hist_fic = {"historical fiction", "historical", "period drama"}
    biography = {"biography", "memoir", "autobiography"}
    self_help = {"self-help", "self improvement", "personal growth"}
    business = {"business", "finance", "economics"}
    travel = {"travel", "adventure", "exploration"}
    cookbook = {"cookbook", "cooking", "recipes"}
    poetry = {"poetry", "verse", "rhyme"}
    drama = {"drama", "play", "theatre"}
    humor = {"humor", "comedy", "funny"}
    children = {"children's literature", "kids", "juvenile"}
    ya = {"young adult literature", "ya", "teen"}
    graphic = {"graphic novels", "comics", "sequential art"}
    psychology = {"psychology", "mind", "mental health", "therapy", "counseling"}
    philosophy = {"philosophy", "ethics", "morality", "metaphysics", "logic"}
    science = {"science", "scientific", "biology", "chemistry", "physics"}
    excited = {"excited", "thrilled", "elated", "eager"}
    happy = {"happy", "joyful", "pleased", "content"}
    angry = {"angry", "outraged", "furious", "irritated"}
    sad = {"sad", "heartbroken", "miserable", "depressed"}
    neutral = {"neutral", "indifferent", "objective", "unbiased"}
    content = {"content", "satisfied", "grateful", "thankful"}
    enthusiastic = {"enthusiastic", "passionate", "zealous", "fervent"}
    surprised = {"surprised", "shocked", "amazed", "astounded"}
    disappointed = {"disappointed", "let down", "unhappy", "dissatisfied"}
    frustrated = {"frustrated", "exasperated", "annoyed", "irritated"}
    proud = {"proud", "accomplished", "confident", "self-assured"}
    jealous = {"jealous", "envious", "covetous", "possessive"}
    hopeful = {"hopeful", "optimistic", "positive", "confident"}
    worried = {"worried", "anxious", "concerned", "uneasy"}
    insecure = {"insecure", "uncertain", "self-doubting", "unconfident"}
    grateful = {"grateful", "thankful", "appreciative", "indebted"}
    calm = {"calm", "relaxed", "serene", "placid"}
    lonely = {"lonely", "isolated", "alone", "desolate"}
    nervous = {"nervous", "apprehensive", "fearful", "timid"}


    genre_set = set()

    for genre in genres:
        if genre.lower() in horror:
            genre_set.add("Horror")
        elif genre.lower() in fantasy:
            genre_set.add("Fantasy")
        elif genre.lower() in sci_fi:
            genre_set.add("Science Fiction")
        elif genre.lower() in romance:
            genre_set.add("Romance")
        elif genre.lower() in mystery:
            genre_set.add("Mystery")
        elif genre.lower() in thriller:
            genre_set.add("Thriller")
        elif genre.lower() in hist_fic:
            genre_set.add("Historical Fiction")
        elif genre.lower() in biography:
            genre_set.add("Biography")
        elif genre.lower() in self_help:
            genre_set.add("Self-help")
        elif genre.lower() in business:
            genre_set.add("Business")
        elif genre.lower() in travel:
            genre_set.add("Travel")
        elif genre.lower() in cookbook:
            genre_set.add("Cookbook")
        elif genre.lower() in poetry:
            genre_set.add("Poetry")
        elif genre.lower() in drama:
            genre_set.add("Drama")
        elif genre.lower() in humor:
            genre_set.add("Humor")
        elif genre.lower() in children:
            genre_set.add("Children's literature")
        elif genre.lower() in ya:
            genre_set.add("Young adult literature")
        elif genre.lower() in graphic:
            genre_set.add("Graphic novels and comics")
        elif genre.lower() in psychology:
            genre_set.add("Psychology")
        elif genre.lower() in philosophy:
            genre_set.add("Philosophy")
        elif genre.lower() in science:
            genre_set.add("Science")
        elif genre.lower() in happy:
            genre_set.add("Happy")
        elif genre.lower() in excited:
            genre_set.add("Excited")
        elif genre.lower() in happy:
            genre_set.add("Happy")
        elif genre.lower() in angry:
            genre_set.add("Angry")
        elif genre.lower() in sad:
            genre_set.add("Sad")
        elif genre.lower() in neutral:
            genre_set.add("Neutral")
        elif genre.lower() in content:
            genre_set.add("Content")
        elif genre.lower() in enthusiastic:
            genre_set.add("Enthusiastic")
        elif genre.lower() in surprised:
            genre_set.add("Surprised")
        elif genre.lower() in disappointed:
            genre_set.add("Disappointed")
        elif genre.lower() in frustrated:
            genre_set.add("Frustrated")
        elif genre.lower() in proud:
            genre_set.add("Proud")
        elif genre.lower() in jealous:
            genre_set.add("Jealous")
        elif genre.lower() in hopeful:
            genre_set.add("Hopeful")
        elif genre.lower() in worried:
            genre_set.add("Worried")
        elif genre.lower() in insecure:
            genre_set.add("Insecure")
        elif genre.lower() in grateful:
            genre_set.add("Grateful")
        elif genre.lower() in calm:
            genre_set.add("Calm")
        elif genre.lower() in lonely:
            genre_set.add("Lonely")
        elif genre.lower() in nervous:
            genre_set.add("Nervous")
        else:
            genre_set.add("Unknown")





    return genre_set


In [14]:
df.drop('Genre', axis=1, inplace=True)
df.drop('Author', axis=1, inplace=True)

## Cleaned Genre

In [15]:
df['Cleaned_Genre'] = df['Genres'].apply(lambda x: get_genre(x))


df.to_csv("../BOOK_DATASET/oooo.csv")
df.head(2)

,Index,ID,BookTitle,Summary,Genres,clean_summary,emotion,Cleaned_Genre
0,1,620,Animal Farm,"Old Major, the old boar on the Manor Farm, calls the animals on the farm for a meeting, where he compares the humans to parasites and teaches the animals a revolutionary song, 'Beasts of England'. When Major dies, two young pigs, Snowball and Napoleon, assume command and turn his dream into a p...","[Roman à clef, Satire, Children's literature, Speculative fiction, Fiction]",old major the old boar on the manor farm calls the animals on the farm for a meeting where he compares the humans to parasites and teaches the animals a revolutionary song beasts of england when major dies two young pigs snowball and napoleon assume command and turn his dream into a philosophy t...,Happy,"{Children's literature, Unknown}"
1,2,843,A Clockwork Orange,"Alex, a teenager living in near-future England, leads his gang on nightly orgies of opportunistic, random ""ultra-violence."" Alex's friends (""droogs"" in the novel's Anglo-Russian slang, Nadsat) are: Dim, a slow-witted bruiser who is the gang's muscle; Georgie, an ambitious second-in-command; and...","[Science Fiction, Novella, Speculative fiction, Utopian and dystopian fiction, Satire, Fiction]",alex a teenager living in near future england leads his gang on nightly orgies of opportunistic random ultra violence alexs friends droogs in the novels anglo russian slang nadsat are dim a slow witted bruiser who is the gangs muscle georgie an ambitious second in command and pete who mostly pla...,Neutral,"{Science Fiction, Unknown}"


### Check the cased if it work

In [16]:
genre = get_genre(["sci-fi"])
print(genre)

{'Science Fiction'}


In [17]:
genre = get_genre(["horror"])
books = df[df['Cleaned_Genre'].apply(lambda x: genre.intersection(x)).astype(bool)]['BookTitle']
print(books.head())



26                     Dracula
37                 Gaudy Night
82                 The Shining
88    The Pit and the Pendulum
89    The Hunt for Red October
Name: BookTitle, dtype: object


### We need to recommend the books based on Ratings ,So we read a another csv which contains Ratings <br><br>So, it will recommend the books based on rating

In [18]:
df2 = pd.read_csv("../BOOK_DATASET/Preprocessed_data.csv")

df2.head(2)

,Unnamed: 0,user_id,location,age,isbn,rating,book_title,book_author,year_of_publication,publisher,img_s,img_m,img_l,Summary,Language,Category,city,state,country
0,0,2,"stockton, california, usa",18.0000,0195153448,0,Classical Mythology,Mark P. O. Morford,2002.0,Oxford University Press,http://images.amazon.com/images/P/0195153448.01.THUMBZZZ.jpg,http://images.amazon.com/images/P/0195153448.01.MZZZZZZZ.jpg,http://images.amazon.com/images/P/0195153448.01.LZZZZZZZ.jpg,"Provides an introduction to classical myths placing the addressed\ntopics within their historical context, discussion of archaeological\nevidence as support for mythical events, and how these themes have\nbeen portrayed in literature, art, ...",en,['Social Science'],stockton,california,usa
1,1,8,"timmins, ontario, canada",34.7439,0002005018,5,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.01.THUMBZZZ.jpg,http://images.amazon.com/images/P/0002005018.01.MZZZZZZZ.jpg,http://images.amazon.com/images/P/0002005018.01.LZZZZZZZ.jpg,"In a small town in Canada, Clara Callan reluctantly takes leave of her\nsister, Nora, who is bound for New York.",en,['Actresses'],timmins,ontario,canada


### So we Filter the books that includes in 1st Dataset and arrange the rating to descending order

### Get a input form user

In [19]:
keywords = {
    "horror": {"horror", "terror", "fear", "suspense", "supernatural"},
    "fantasy": {"fantasy", "magic", "mythical", "legendary", "enchanted"},
    "sci_fi": {"science fiction", "sci-fi", "space"},
    "romance": {"romance", "love", "heartbreak", "passion"},
    "mystery": {"mystery", "detective", "crime", "sleuth"},
    "thriller": {"thriller", "suspense", "action"},
    "hist_fic": {"historical fiction", "historical", "period drama"},
    "biography": {"biography", "memoir", "autobiography"},
    "self_help": {"self-help", "self improvement", "personal growth"},
    "business": {"business", "finance", "economics"},
    "travel": {"travel", "adventure", "exploration"},
    "cookbook": {"cookbook", "cooking", "recipes"},
    "poetry": {"poetry", "verse", "rhyme"},
    "drama": {"drama", "play", "theatre"},
    "humor": {"humor", "comedy", "funny"},
    "children": {"children's literature", "kids", "juvenile"},
    "ya": {"young adult literature", "ya", "teen"},
    "graphic": {"graphic novels", "comics", "sequential art"},
    "psychology": {"psychology", "mind", "mental health", "therapy", "counseling"},
    "philosophy": {"philosophy", "ethics", "morality", "metaphysics", "logic"},
    "science": {"science", "scientific", "biology", "chemistry", "physics"},
    "excited": {"excited", "thrilled", "elated", "eager"},
    "Happy": {"happy", "joyful", "pleased", "content"},
    "angry": {"angry", "outraged", "furious", "irritated"},
    "Sad": {"sad", "heartbroken", "miserable", "depressed"},
    "Neutral": {"neutral", "indifferent", "objective", "unbiased"},
    "Content": {"content", "satisfied", "grateful", "thankful"},
    "Enthusiastic": {"enthusiastic", "passionate", "zealous", "fervent"},
    "Surprised": {"surprised", "shocked", "amazed", "astounded"},
    "Disappointed": {"disappointed", "let down", "unhappy", "dissatisfied"},
    "Frustrated": {"frustrated", "exasperated", "annoyed", "irritated"},
    "Proud": {"proud", "accomplished", "confident", "self-assured"},
    "Jealous": {"jealous", "envious", "covetous", "possessive"},
    "Hopeful": {"hopeful", "optimistic", "positive", "confident"},
    "Worried": {"worried", "anxious", "concerned", "uneasy"},
    "Insecure": {"insecure", "uncertain", "self-doubting", "unconfident"},
    "Grateful": {"grateful", "thankful", "appreciative", "indebted"},
    "Calm": {"calm", "relaxed", "serene", "placid"},
    "Lonely": {"lonely", "isolated", "alone", "desolate"},
    "Nervous": {"nervous", "apprehensive", "fearful", "timid"}
}


In [20]:
emotions = ["Excited", "Happy", "Angry", "Sad", "Neutral", "Content", "Enthusiastic", "Surprised",
"Disappointed", "Frustrated", "Proud", "Jealous", "Hopeful", "Worried", "Insecure",
"Grateful", "Calm", "Lonely", "Nervous"]

In [21]:
# Define the set of emotions you want to filter by
target_emotion = 'Neutral'

# Filter the 'BookTitle' column based on the presence of the target emotion
books = df[df['emotion'].apply(lambda x: x in target_emotion)]['BookTitle']
books


1                    A Clockwork Orange
2                            The Plague
4                  A Fire Upon the Deep
5        All Quiet on the Western Front
6                  A Wizard of Earthsea
                      ...              
16549                    The Third Lynx
16551                    Remote Control
16555                 Transfer of Power
16556                           Decoded
16558                         Poor Folk
Name: BookTitle, Length: 8732, dtype: object

In [34]:
input_str = input("Input :- ")
selected_keyword = None
for keyword, values in keywords.items():
    for value in values:
        if value in input_str.lower():
            selected_keyword = keyword
            break
    if selected_keyword:
        break
else:
    selected_keyword = 'default_value'  # Set some default value if no keyword is found

genre = get_genre([selected_keyword])
if set(genre) <= set(emotions):
    books = df[df['emotion'].apply(lambda x: x in selected_keyword)]['BookTitle']
else:
    books = df[df['Cleaned_Genre'].apply(lambda x: set(genre).intersection(x)).astype(bool)]['BookTitle']

# Filter df2 to only include books in the "books" list
df2_filtered = df2[df2['book_title'].isin(books)]

# Sort df2 by descending rating
df2_sorted = df2_filtered.sort_values(by='rating', ascending=False)

# Display the book titles in the sorted dataframe
print("-------------------------------")
print(df2_sorted['book_title'].head(5))
print("-------------------------------")


Input :- Romantic
-------------------------------
648370    The Ghost Writer
245599       Cold Mountain
245625       Cold Mountain
245616       Cold Mountain
64438       The Lake House
Name: book_title, dtype: object
-------------------------------


In [35]:
genre = get_genre([selected_keyword])
if set(genre) <= set(emotions):
    print('true')

In [36]:
books = df[df['emotion'].apply(lambda x: set(genre).intersection(x)).astype(bool)]['BookTitle']
books

Series([], Name: BookTitle, dtype: object)

In [33]:
# # Merge the two datasets on book title
# merged_df = pd.merge(df, df2, on='book_title')

# # Filter the merged dataset to only include rows with selected emotion keywords
# filtered_df = merged_df[merged_df['emotion'].isin(selected_keyword)]

# # Sort the filtered dataset by descending rating
# sorted_df = filtered_df.sort_values(by='rating', ascending=False)

# # Get top 5 book titles
# top_books = sorted_df['book_title'].iloc[:5]


### At Finally , it will recommend top 5 books

In [37]:
selected_books = []
top_5_books = []
for book_title in df2_sorted['book_title']:
    if book_title not in selected_books:
        top_5_books.append(book_title)
        selected_books.append(book_title)
        if len(top_5_books) == 5:
            break
print(f"Recommended books for {genre}")
for item in top_5_books:
    print('- ' + item)
# print(top_5_books)


Recommended books for {'Unknown'}
- The Ghost Writer
- Cold Mountain
- The Lake House
- The Assignment
- 2nd Chance


In [ ]:
df.to_csv('output.csv', index=False)
